## Provide list of paths for edits and run trials -> (maybe analyze results and add to CSV)

In [1]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
from datetime import datetime
import pandas as pd

In [2]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, informal_log, list_to_dict
from utils.model_utils import prepare_device
from parse_config import ConfigParser
# from data_loader import data_loaders
import datasets.datasets as module_data
import model.model as module_arch
from utils.knn_utils import load_and_analyze_knn
from utils.analysis_utils import combine_results
from edit_knn import main as edit

In [3]:
# Obtain timestamp
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

In [4]:
# Define constants, paths
config_path = 'configs/copies/edit_experiments/cov_identity_trials.json'
class_list_path = 'metadata/cinic-10/class_names.txt'
# key_image_id = 'dog-train-n02114712_211'
target_class_name = 'dog'

analyze_in_edit = True
# paths_dir = os.path.join('paths', 'edits', key_image_id)
paths_dir = os.path.join('paths', 'edits', target_class_name, '0113_155402')

key_image_paths = read_lists(os.path.join(paths_dir, 'key_images.txt'))
value_image_paths = read_lists(os.path.join(paths_dir, 'value_images.txt'))
n_trials = len(value_image_paths)
assert len(key_image_paths) == n_trials

print("First key image path: {}".format(key_image_paths[0]))
print("First value image path: {}".format(value_image_paths[0]))

First key image path: data/cinic-10-imagenet/train/dog/n02114712_211.png
First value image path: saved/segmentations/dog/dog-train-n02114712_211/modified_images/felzenszwalb_gaussian_0.png


In [5]:
# Load config file
config_dict = read_json(config_path)
# Load class list and obtain target class idx
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)
target_class_idx = class_idx_dict[target_class_name]

# Set K
K = config_dict['editor']['K']

device, device_ids = prepare_device(config_dict['n_gpu'])

In [6]:
# Load datasets
data_loader_args = dict(config_dict["data_loader"]["args"])
dataset_args = dict(config_dict["dataset_args"])

# Create validation data loader
val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Create data loader for covariance matrix
covariance_image_paths = read_lists(config_dict['covariance_dataset']['images'])
covariance_labels = read_lists(config_dict['covariance_dataset']['labels'])

covariance_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=covariance_image_paths,
        labels=covariance_labels,
        **dataset_args
    ),
    **data_loader_args
)

### Create log and save paths

In [7]:
# create log path to store the paths to each trial
save_root = config_dict['trainer']['save_dir']
save_trials_path = os.path.join(save_root, config_dict['name'], timestamp, 'trial_paths.txt')
if os.path.exists(save_trials_path):
    # os.remove(save_trials_path)
    print("Path {} already exists. Aborting.".format(save_trials_path))
else:
    progress_report_path = os.path.join(save_root, config_dict['name'], timestamp, 'progress_report.txt')
    if os.path.exists(progress_report_path):
        os.remove(progress_report_path)
    print("Saving path to directories for each trial to {}".format(save_trials_path))
    print("Printing progress reports to {}".format(progress_report_path))

Saving path to directories for each trial to saved/edit/trials/identity_cov/CINIC10_ImageNet-VGG_16/0117_201313/trial_paths.txt
Printing progress reports to saved/edit/trials/identity_cov/CINIC10_ImageNet-VGG_16/0117_201313/progress_report.txt


### Ensure all paths for keys and values exist

In [8]:
non_existent_key_paths = []
non_existent_value_paths = []
for key_path, value_path in zip(key_image_paths, value_image_paths):
    if not os.path.exists(key_path):
        non_existent_key_paths.append(key_path)
    if not os.path.exists(value_path):
        non_existent_value_paths.append(value_path)

if len(non_existent_key_paths) > 0:
    raise ValueError("Following paths are non existent: {}".format(non_existent_key_paths))
    
if len(non_existent_value_paths) > 0:
    raise ValueError("Following paths are non existent: {}".format(non_existent_value_paths))

## Run edit for each modified image

In [11]:
for idx, (key_path, value_path) in enumerate(zip(key_image_paths, value_image_paths)):
    split = os.path.basename(os.path.dirname(os.path.dirname(key_path)))
    class_name = os.path.basename(os.path.dirname(key_path))
    file_name = os.path.basename(key_path).split(".")[0]
    key_image_id = "{}-{}-{}".format(class_name, split, file_name)
    # Print Progress
    informal_log("({}) Starting Trial {}/{}...".format(datetime.now().strftime(r'%m%d_%H%M%S'), idx + 1, n_trials), progress_report_path)
    
    # Create run id 
    value_image_id = os.path.splitext(os.path.basename(value_path))[0]
    run_id = os.path.join(timestamp, key_image_id, value_image_id)
    informal_log("Current run_id: {}".format(run_id), progress_report_path)
    
    # Read config file as json and make updates to key and value paths
    config_dict = read_json(config_path)
    config_dict['editor'].update({
        'key_image_path': key_path,
        'value_image_path': value_path
    })
    
    # Create config object
    config = ConfigParser(config_dict, run_id=run_id)
    
    # Log the current trial path
    informal_log(os.path.dirname(config.save_dir), save_trials_path)
    
    informal_log("Calling edit()...", progress_report_path)
    
    edit(
        config=config,
        val_paths_data_loader=val_paths_data_loader,
        covariance_data_loader=covariance_data_loader,
        do_analyze_knn=analyze_in_edit)
    '''
    Repeat for identity matrix as covariance matrix
    '''
    # Create config object
    config = ConfigParser(config_dict, run_id=os.path.join(run_id, 'identity'))
    
    config.config['covariance_dataset'] = {"name": ""}
    # Log the current trial path
    informal_log(os.path.dirname(config.save_dir), save_trials_path)
    
    informal_log("Calling edit()...", progress_report_path)
    
    edit(
        config=config,
        val_paths_data_loader=val_paths_data_loader,
        covariance_data_loader=None,
        do_analyze_knn=analyze_in_edit)
    
    # Print progress
    informal_log("Finished trial {}/{}. Results saved to {}".format(idx + 1, n_trials, os.path.dirname(config.save_dir)),
                progress_report_path)


(0117_201637) Starting Trial 1/37...
Current run_id: 0117_201313/dog-train-n02114712_211/felzenszwalb_gaussian_0
saved/edit/trials/identity_cov/CINIC10_ImageNet-VGG_16/0117_201313/dog-train-n02114712_211/felzenszwalb_gaussian_0
Calling edit()...
Created ModelWrapperSanturkar model with 33646666 trainable parameters
Restored weights from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg16_bn.pt
Using passed in data loader for validation.
Key images: data/cinic-10-imagenet/train/dog/n02114712_211.png
Value images: saved/segmentations/dog/dog-train-n02114712_211/modified_images/felzenszwalb_gaussian_0.png
Masks: None
Prepared data for editing
Performing pre-edit metric calculations on validation set.


100%|██████████████████████████████████████████| 274/274 [00:07<00:00, 34.46it/s]

Pre-edit metrics: {'TP': array([6142, 5382, 5203, 3842, 4270, 2990, 5659, 5136, 4820, 4679]), 'TN': array([60115, 61110, 59441, 59902, 60157, 61756, 61050, 61747, 61643,
       61202]), 'FPs': array([2885, 1890, 3559, 3098, 2843, 1244, 1950, 1253, 1357, 1798]), 'FNs': array([ 858, 1618, 1797, 3158, 2730, 4010, 1341, 1864, 2180, 2321]), 'accuracy': 0.6874714285714286, 'per_class_accuracy': array([0.94652857, 0.94988571, 0.92348571, 0.91062857, 0.92038571,
       0.92494286, 0.95298571, 0.95547143, 0.94947143, 0.94115714]), 'per_class_accuracy_mean': 0.9374942857142857, 'precision': array([0.68040323, 0.74009901, 0.5938142 , 0.55360231, 0.60030929,
       0.706188  , 0.74372454, 0.80388167, 0.78031407, 0.72240235]), 'precision_mean': 0.6924738665961387, 'recall': array([0.87742857, 0.76885714, 0.74328571, 0.54885714, 0.61      ,
       0.42714286, 0.80842857, 0.73371429, 0.68857143, 0.66842857]), 'recall_mean': 0.6874714285714286, 'f1': array([0.7664566 , 0.75420404, 0.66019541, 0.551219

Found precomputed cov matrices, returning...
tensor([[ 1.0000e+00, -1.8504e-05, -3.8476e-05, -1.9159e-05, -4.0293e-05,
         -3.1667e-05, -2.4137e-05, -2.9244e-05, -4.0682e-05, -2.2710e-05],
        [-1.8504e-05,  1.0000e+00, -2.8878e-05, -1.2513e-05, -2.8300e-05,
         -2.1209e-05, -1.7897e-05, -2.0580e-05, -2.4006e-05, -1.4785e-05],
        [-3.8476e-05, -2.8878e-05,  1.0000e+00, -2.8741e-05, -6.1447e-05,
         -4.6148e-05, -3.9829e-05, -4.5861e-05, -4.7019e-05, -3.5061e-05],
        [-1.9159e-05, -1.2513e-05, -2.8741e-05,  1.0000e+00, -2.7411e-05,
         -2.1258e-05, -1.7047e-05, -1.9973e-05, -2.4082e-05, -1.5679e-05],
        [-4.0293e-05, -2.8300e-05, -6.1447e-05, -2.7411e-05,  1.0000e+00,
         -4.6443e-05, -5.1575e-05, -4.8059e-05, -5.6685e-05, -3.6807e-05],
        [-3.1667e-05, -2.1209e-05, -4.6148e-05, -2.1258e-05, -4.6443e-05,
          1.0000e+00, -2.6984e-05, -3.3795e-05, -4.0129e-05, -2.3401e-05],
        [-2.4137e-05, -1.7897e-05, -3.9829e-05, -1.7047e-05, 

tensor(0.0044, device='cuda:0', grad_fn=<DivBackward0>): 100%|█| 40000/40000 [00:

...

Found precomputed cov matrices, returning...
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0')


tensor(0.0144, device='cuda:0', grad_fn=<DivBackward0>): 100%|█| 40000/40000 [00:

Loss (orig, final): 0.11681278049945831 0.014357169158756733
L2 norm of weight change: 0.3410739004611969
Performing post-edit metric calculations on validation set.



100%|██████████████████████████████████████████| 274/274 [00:09<00:00, 28.76it/s]

Post-edit metrics: {'TP': array([6024, 5703, 5245, 3704, 3600, 3902, 5748, 3916, 5028, 4242]), 'TN': array([60574, 60322, 59333, 60083, 61450, 58918, 60782, 62742, 61148,
       61760]), 'FPs': array([2426, 2678, 3667, 2917, 1550, 4082, 2218,  258, 1852, 1240]), 'FNs': array([ 976, 1297, 1755, 3296, 3400, 3098, 1252, 3084, 1972, 2758]), 'accuracy': 0.6730285714285714, 'per_class_accuracy': array([0.9514    , 0.94321429, 0.92254286, 0.91124286, 0.92928571,
       0.89742857, 0.95042857, 0.95225714, 0.94537143, 0.94288571]), 'per_class_accuracy_mean': 0.9346057142857143, 'precision': array([0.71289941, 0.68046772, 0.58853232, 0.55943211, 0.69902913,
       0.48872745, 0.72156666, 0.93818879, 0.73081395, 0.77380518]), 'precision_mean': 0.6893462720065023, 'recall': array([0.86057143, 0.81471429, 0.74928571, 0.52914286, 0.51428571,
       0.55742857, 0.82114286, 0.55942857, 0.71828571, 0.606     ]), 'recall_mean': 0.6730285714285713, 'f1': array([0.77980583, 0.74156427, 0.65925088, 0.54386

In [15]:
save_trials_path = 'saved/edit/trials/identity_cov/CINIC10_ImageNet-VGG_16/0117_201313/trial_paths.txt'
trial_dirs = read_lists(save_trials_path)
knn_analysis_filename = 'knn_analysis_results.pth'

In [16]:
try:
    if not os.path.exists(save_trials_path):
        print("Path {} does not exist".format(save_trials_path))
    else:
        print("Obtaining trial paths from {}".format(save_trials_path))
except:
    print("Need to define save_trials_path.")



Obtaining trial paths from saved/edit/trials/identity_cov/CINIC10_ImageNet-VGG_16/0117_201313/trial_paths.txt


## For each trial pair, compare the difference in all metrics and report the average across all data

In [24]:
# Sanity check of even number of paths
local_path = os.path.join('models', '{}_edit_metrics.pth')
assert len(trial_dirs) % 2 == 0
n_trials = len(trial_dirs) // 2
differences = {}
for trial_idx in range(n_trials):
    control_dir = trial_dirs[2 * trial_idx]
    experimental_dir = trial_dirs[2 * trial_idx + 1]
    assert experimental_dir == os.path.join(control_dir, 'identity')
    control_pre = torch.load(os.path.join(control_dir, local_path.format('pre')))
    control_post = torch.load(os.path.join(control_dir, local_path.format('post')))
    
    exp_pre = torch.load(os.path.join(experimental_dir, local_path.format('pre')))
    exp_post = torch.load(os.path.join(experimental_dir, local_path.format('post')))
    
    keys = control_pre.keys()
    # Assert pre edit metrics are the same
    for key in keys:
        try:
            assert control_pre[key] == exp_pre[key]
        except: 
            assert (control_pre[key] == exp_pre[key]).all()
        if trial_idx == 0:
            differences[key] = [exp_post[key] - exp_pre[key]]
        else: 
            differences[key].append(exp_post[key] - exp_pre[key])

print(differences)

{'TP': [array([ -21, -106,  185,  157,   42,  125, -678,   40,   75,  -63]), array([ -21, -106,  185,  157,   42,  125, -678,   40,   75,  -63]), array([ -36,  -42,  148,  205,  -63,  251, -606,   51,   81,  -85]), array([ -108,   124,   177,  -283,  -522,  1081, -1464,   120,    13,
        -306]), array([ -28,  -28,   87,  287, -225,  133, -277,   36,    2,    2]), array([  -2,  -55,  131,  128,   19,  128, -526,   38,   98,  -50]), array([ -49,  -67,  -83,  431,  -59,  304, -576,   34,  101,  -42]), array([  -50,    22, -2582,   313,   218,   468,    56,   -73,   212,
         -12]), array([ -44,   55,  -21, -124,  -27,  105,   36,   18,   59,   10]), array([ -54,  126,  349, -478,  -92,  222,  -30,  -35,  -95, -270]), array([-41,  24,  63, -20, -27, 158, -47, -27,  34, -91]), array([ -119,   211,   595, -1519,  -126,  -136,   -56,    53,   -17,
        -195]), array([-41,  21,  57,   5, -34, 150, -47, -36,  37, -87]), array([ -19,  149,  482, -813,  -68,  -12,  -67,  121, -298, -32

## Analyze KNN

In [23]:
## Process KNN results for each
n_trials = len(trial_dirs)
n_log = n_trials // 10 + 1  # log every 10%
progress_report_path = os.path.join(os.path.dirname(save_trials_path), 'progress_report_analysis.txt')

informal_log("Starting KNN analysis...", progress_report_path)
for trial_idx, trial_dir in tqdm(enumerate(trial_dirs)):
    # if trial_idx % n_log == 0:
    informal_log("Processing {}/{} trials. Currently processing {}".format(
        trial_idx+1, n_trials, os.path.basename(trial_dir)), progress_report_path)
                     
    results_save_dir = os.path.join(trial_dir, 'models')
    load_and_analyze_knn(
        restore_dir=results_save_dir,
        pre_edit_knn_path=os.path.join(results_save_dir, 'pre_edit_{}-nn.pth'.format(K)),
        post_edit_knn_path=os.path.join(results_save_dir, 'post_edit_{}-nn.pth'.format(K)),
        knn_analysis_filename=knn_analysis_filename,
        target_class_idx=target_class_idx,
        class_list=class_list,
        progress_report_path=progress_report_path,
        save_images=False,
        save_plots=True)
    


Starting KNN analysis...


0it [00:00, ?it/s]

Processing 1/37 trials. Currently processing felzenszwalb_gaussian_0
Analyzing KNN results from saved/edit/trials/CINIC10_ImageNet-VGG_16/0112_121958/dog-train-n02114712_211/felzenszwalb_gaussian_0/models
Logging and saving visualizations to saved/edit/trials/CINIC10_ImageNet-VGG_16/0112_121958/dog-train-n02114712_211/felzenszwalb_gaussian_0/models/knn_visualizations/knn_analysis_log.txt
Saving results to saved/edit/trials/CINIC10_ImageNet-VGG_16/0112_121958/dog-train-n02114712_211/felzenszwalb_gaussian_0/models/knn_analysis_results.pth


0it [00:00, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'saved/edit/trials/CINIC10_ImageNet-VGG_16/0112_121958/dog-train-n02114712_211/felzenszwalb_gaussian_0/models/edited_model.pth'

## Convert to CSV for all trials

In [45]:
data = []
# Iterate through all trials
for trial_idx, trial_dir in tqdm(enumerate(trial_dirs)):
    # Obtain key ID from path
    key_id = os.path.basename(os.path.dirname(trial_dir))
    id_class = key_id.split('-')[0]
    if id_class not in class_list:
        raise ValueError("Invalid key_id {}".format(key_id))
    # Obtain value ID from path
    val_id = os.path.basename(trial_dir)
    # Join to make a data ID
    data_id = os.path.join(key_id, val_id)
    
    # Load results from knn, pre-edit metrics, and post-edit metrics
    restore_dir = os.path.join(trial_dir, 'models')
    knn_analysis_results = torch.load(os.path.join(restore_dir, knn_analysis_filename))
    pre_edit_metrics = torch.load(os.path.join(restore_dir, 'pre_edit_metrics.pth'))
    post_edit_metrics = torch.load(os.path.join(restore_dir, 'post_edit_metrics.pth'))
    
    # Combine results into one dictionary
    combined_results = combine_results(
        data_id=data_id,
        knn_analysis=knn_analysis_results,
        pre_edit_metrics=pre_edit_metrics,
        post_edit_metrics=post_edit_metrics)
    
    # Save column headers in first trial run
    if trial_idx == 0:
        column_headers = list(combined_results.keys())
    # Convert results to np.array & append to list
    combined_results = np.expand_dims(np.array(list(combined_results.values())), axis=0)
    data.append(combined_results)

# Convert data from list of np.arrays -> pd.DataFrame    
data = np.concatenate(data, axis=0)
df = pd.DataFrame(data, columns=column_headers)


2008it [00:16, 120.03it/s]


In [46]:
# Save to CSV
csv_save_path = os.path.join(os.path.dirname(save_trials_path), 'results_table.csv')
df.to_csv(csv_save_path)
print("Saved CSV to {}".format(csv_save_path))

Saved CSV to saved/edit/trials/CINIC10_ImageNet-VGG_16/0113_160154/results_table.csv
